In [1]:
#Import libraries

import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import sys
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import random

import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import models
import torchvision.utils
import torchvision.datasets as dsets
import torchvision.transforms as transforms

#from torchattacks import PGD, FGSM, FFGSM
import rep_transformations as rt
from no_attack import NO_ATTACK
from stochastic_attack import STOCHASTIC_ATTACK
from fgsm_ import FGSM_
from ffgsm_ import FFGSM_
from slide_ import SLIDE_
from pgd_ import PGD_
from pgdl2_ import PGDL2_
from data_transformations import DctBlurry

In [2]:
#Show CUDA settings

print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
  
cuda_id = torch.cuda.current_device()
print(f"ID of current CUDA device: {cuda_id}")

cuda_name = torch.cuda.get_device_name(cuda_id)
print(f"Name of current CUDA device: {cuda_name}\n")
print(f"CUDA devide properties: {torch.cuda.get_device_properties(0)}")

Is CUDA supported by this system? True
CUDA version: 11.1
ID of current CUDA device: 0
Name of current CUDA device: NVIDIA GeForce RTX 3060 Laptop GPU

CUDA devide properties: _CudaDeviceProperties(name='NVIDIA GeForce RTX 3060 Laptop GPU', major=8, minor=6, total_memory=6143MB, multi_processor_count=30)


In [3]:
#Downloading Fashion MNIST Dataset
train_list_transforms = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        #DctBlurry(threshold=5),
    ])

test_list_transforms = transforms.Compose([
        transforms.ToTensor(),
    ])

fashion_mnist_train = dsets.FashionMNIST(root='./data1/',
                          train=True,
                          transform=train_list_transforms,
                          download=False)

fashion_mnist_aux = dsets.FashionMNIST(root='./data1/',
                         train=False,
                         transform=test_list_transforms,
                         download=False)

fashion_mnist_test, fashion_mnist_valid = torch.utils.data.random_split(fashion_mnist_aux, [5000, 5000], generator=torch.Generator().manual_seed(42))

In [4]:
#Initialize DataLoader

batch_size = 128
torch.manual_seed(0)
n_workers = 5

train_loader  = torch.utils.data.DataLoader(dataset=fashion_mnist_train,
                                           batch_size=batch_size,
                                           shuffle=42, num_workers=n_workers)

test_loader = torch.utils.data.DataLoader(dataset=fashion_mnist_test,
                                         batch_size=batch_size,
                                         shuffle=42, num_workers=n_workers)

valid_loader = torch.utils.data.DataLoader(dataset=fashion_mnist_valid,
                                         batch_size=batch_size,
                                         shuffle=42, num_workers=n_workers)

In [5]:
#Initialize transformations

id_transf = rt.Identity()
fft_transf = rt.FFT()
dct_transf = rt.DCT()
jpeg_transf = rt.JPEG()

list_transf = [id_transf, fft_transf, dct_transf, jpeg_transf]

In [6]:
#Get model and optimizer

def generate_model(pretrained=True, lr=0.001):

    #Init model and optimizer
    model = torchvision.models.resnet50(pretrained=True).cuda() #CNN.cuda() to resnet 50
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    #Change model architecture 
    model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
    model.fc = nn.Linear(2048, 10, bias=True)
    
    #Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if torch.cuda.is_available():
        model.to(device)
        
    return model, optimizer

In [7]:
#modified test function with 3 attacks and toggle between Pixel and DCT representation.Default is Pixel

def get_accuracy(model, test_loader, atk):
    
    model.eval()
    correct = 0
    total = 0    

    for images, labels in test_loader:

        images = atk(images, labels).cuda()              
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels.cuda()).sum()

    return 100 * float(correct) / total

In [8]:
#train function with PGD attack and toggle between DCT and Pixel representation .Default is Pixel

def train_(model, train_loader, optimizer, loss, atk, num_epochs=5):

    model.train()

    for epoch in range(num_epochs):

        total_batch = len(fashion_mnist_train) // batch_size
        
        for i, (batch_images, batch_labels) in enumerate(train_loader):

           
            Y = batch_labels.cuda()
            X = atk(batch_images, batch_labels).cuda()

            pre = model(X)
            cost = loss(pre, Y)

            optimizer.zero_grad()
            cost.backward()
            optimizer.step()

            if (i+1) % 100 == 0:
                print(f'Train. Atk: {atk.attack}, Epoch [{epoch+1}/{num_epochs}], lter [{i+1}/{total_batch}], Loss: {cost.item()}')
    return cost

In [9]:
def round_robin(model, train_loader, optimizer, loss, atks_list, num_epochs=5):

    model.train() 
    num_attacks = len(atks_list)
    total_batch = len(fashion_mnist_train) // batch_size

    for epoch in range(num_epochs): #+1 to round up

        count = 0

        for i, (batch_images, batch_labels) in enumerate(train_loader):

            atk = atks_list[count]
            Y = batch_labels.cuda()
            X = atk(batch_images, batch_labels).cuda()

            pre = model(X)
            cost = loss(pre, Y)

            optimizer.zero_grad()
            cost.backward()
            optimizer.step()

            count += 1
            count %= num_attacks

            if (i+1) % 100 == 0:
                print(f'RR. Epoch [{epoch+1}/{num_epochs}], lter [{i+1}/{total_batch}], Loss: {cost.item()}')
    return cost

In [10]:
def greedy(model, train_loader, valid_loader, optimizer, loss, atks_list, num_epochs=5):

    num_attacks = len(atks_list)
    total_batch = len(fashion_mnist_train) // batch_size

    for epoch in range(num_epochs):

        loss_list = [0]*num_attacks

        model.eval()

        for i, (batch_images, batch_labels) in enumerate(valid_loader):

            Y = batch_labels.cuda()
            current_batch_size = Y.shape[0]

            for i, atk in enumerate(atks_list):

                X = atk(batch_images, batch_labels).cuda()

                pre = model(X)
                cost = loss(pre, Y)

                loss_list[i] += float(cost)/current_batch_size

        max_loss_arg = np.argmax(np.array(loss_list))
        atk = atks_list[max_loss_arg]

        print(f"Greedy. Epoch [{epoch+1}/{num_epochs}], Worst loss attack: {atk.attack}, loss: {loss_list[max_loss_arg]}")

        model.train()

        for i, (batch_images, batch_labels) in enumerate(train_loader):

            Y = batch_labels.cuda()
            X = atk(batch_images, batch_labels).cuda()

            pre = model(X)
            cost = loss(pre, Y)

            optimizer.zero_grad()
            cost.backward()
            optimizer.step()

            if (i+1) % 100 == 0:
                print(f'Greedy. Epoch [{epoch+1}/{num_epochs}], lter [{i+1}/{total_batch}], Loss: {cost.item()}')
    return cost

In [11]:
def multivariate_probability(model, train_loader, valid_loader, optimizer, loss, atks_list, num_epochs=5):

    num_attacks = len(atks_list)
    total_batch = len(fashion_mnist_train) // batch_size
    w_list = [1]*num_attacks
    eta = 0.1

    for epoch in range(num_epochs):

        loss_list = [0]*num_attacks
        model.eval()

        for i, (batch_images, batch_labels) in enumerate(valid_loader):

            Y = batch_labels.cuda()
            current_batch_size = Y.shape[0]

            for i, atk in enumerate(atks_list):

                X = atk(batch_images, batch_labels).cuda()

                pre = model(X)
                cost = loss(pre, Y)

                loss_list[i] += float(cost)/current_batch_size

        loss_list = np.array(loss_list)
        total_loss = loss_list.sum()
        prob_list = np.cumsum(loss_list)
        prob_list /= total_loss

        log = f"MP. Epoch [{epoch+1}/{num_epochs}], atks: "
        for i, atk in enumerate(atks_list):
            log += f"({atk.attack},{prob_list[i]}) "

        print(log)

        model.train()

        for i, (batch_images, batch_labels) in enumerate(train_loader):

            rand = random.random()
            atk_idx = 0
            for j in range(1,num_attacks):
                if rand > prob_list[j]:
                    atk_idx = j
                else:
                    break

            atk = atks_list[atk_idx]
            Y = batch_labels.cuda()
            X = atk(batch_images, batch_labels).cuda()

            pre = model(X)
            cost = loss(pre, Y)

            optimizer.zero_grad()
            cost.backward()
            optimizer.step()

            if (i+1) % 100 == 0:
                print(f'MP. Epoch [{epoch+1}/{num_epochs}], lter [{i+1}/{total_batch}], Loss: {cost.item()}')
    return cost

In [12]:
def get_model_acc(model, test_loader, atks_list):
    
    model_acc = {}
    
    for atk in atks_list:
        model_atk_acc = get_accuracy(model, test_loader, atk)
        model_acc[atk.attack] = model_atk_acc
        
    return model_acc

In [13]:
#Define attacks_list generation

def generate_atks(model_atk, eps=0.3):
    
    atks_list = []
    atks_list.append(NO_ATTACK(model_atk))
    atks_list.append(STOCHASTIC_ATTACK(model_atk, eps=eps))
    atks_list.append(FGSM_(model_atk, eps=eps))
    atks_list.append(PGD_(model_atk, eps=eps))
    atks_list.append(PGD_(model_atk, eps=eps, transf=dct_transf))
    atks_list.append(PGD_(model_atk, eps=eps, transf=fft_transf))
        
    return atks_list

In [23]:
def get_model_avg(n_models=1, procedure="None", num_epochs=5, eps=0.3):

    loss = nn.CrossEntropyLoss()
    iters_result = {}
    
    for i in range(n_models):
        model, optim = generate_model()
        atks_model = generate_atks(model, eps=eps)
        
        if iters_result == {}:
            for atk in atks_model:
                iters_result[atk.attack] = []

        cost0=train_(model,train_loader,optim,loss,atks_model[0], num_epochs=num_epochs)

        if procedure == "RR":
            cost1 = round_robin(model,train_loader,optim,loss,atks_model, num_epochs=num_epochs)
        elif procedure == "Greedy":
            cost1 = greedy(model,train_loader,valid_loader,optim,loss,atks_model, num_epochs=num_epochs)
        elif procedure == "MP":
            cost1 = multivariate_probability(model,train_loader,valid_loader,optim,loss,atks_model, num_epochs=num_epochs)
        elif procedure == "PGD":
            atk_pgd = PGD_(model, eps=eps, steps=20)
            cost1 = train_(model,train_loader,optim,loss,atk_pgd, num_epochs=num_epochs)
        elif procedure == "PGD_DCT":
            atk_pgd_dct = PGD_(model, eps=eps, steps=20, transf=dct_transf)
            cost1 = train_(model,train_loader,optim,loss,atk_pgd_dct, num_epochs=num_epochs)
        elif procedure == "PGD_FFT":
            atk_pgd_fft = PGD_(model, eps=eps, steps=20, transf=fft_transf)
            cost1 = train_(model,train_loader,optim,loss,atk_pgd_fft, num_epochs=num_epochs)
        
        acc = get_model_acc(model, test_loader, atks_model)
        
        for (key, value) in acc.items():
            temp = iters_result[key]
            temp.append(value)
            iters_result[key] = temp
            
        print(f'Model average. Iter: {i+1}/{n_models}.')
            
    results = {}
    for (key, values) in iters_result.items():
        aux_array = np.array(values)
        mean = np.mean(aux_array)
        std = np.std(aux_array)
        results[key] = str(round(mean, 2))+"+"+str(round(std, 2))
    
    return results

In [15]:
raise Exception('Stop cell')

Exception: Stop cell

In [20]:
#Generate accuracy df/table
std_acc = get_model_avg(n_models=1)

Train. Atk: NO_ATTACK, Epoch [1/5], lter [100/468], Loss: 0.513713002204895
Train. Atk: NO_ATTACK, Epoch [1/5], lter [200/468], Loss: 0.3376750648021698
Train. Atk: NO_ATTACK, Epoch [1/5], lter [300/468], Loss: 0.3800686299800873
Train. Atk: NO_ATTACK, Epoch [1/5], lter [400/468], Loss: 0.3380638659000397
Train. Atk: NO_ATTACK, Epoch [2/5], lter [100/468], Loss: 0.2609924077987671
Train. Atk: NO_ATTACK, Epoch [2/5], lter [200/468], Loss: 0.31914377212524414
Train. Atk: NO_ATTACK, Epoch [2/5], lter [300/468], Loss: 0.4122563898563385
Train. Atk: NO_ATTACK, Epoch [2/5], lter [400/468], Loss: 0.2219519019126892
Train. Atk: NO_ATTACK, Epoch [3/5], lter [100/468], Loss: 0.31166574358940125
Train. Atk: NO_ATTACK, Epoch [3/5], lter [200/468], Loss: 0.38975754380226135
Train. Atk: NO_ATTACK, Epoch [3/5], lter [300/468], Loss: 0.29448363184928894
Train. Atk: NO_ATTACK, Epoch [3/5], lter [400/468], Loss: 0.2129490077495575
Train. Atk: NO_ATTACK, Epoch [4/5], lter [100/468], Loss: 0.2267155349254

In [21]:
df = pd.DataFrame(columns = list(std_acc.keys()))
df = df.append(std_acc, ignore_index=True)
df

C:\Users\omen1\AppData\Local\Temp\ipykernel_6924\326607975.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(std_acc, ignore_index=True)


,NO_ATTACK,RAND_ID,FGSM_ID,PGD_ID,PGD_DCT,PGD_FFT
0,89.94+0.0,58.22+0.0,4.06+0.0,0.0+0.0,0.0+0.0,3.56+0.0


In [24]:
std_acc = get_model_avg(n_models=1, procedure='PGD')

Train. Atk: NO_ATTACK, Epoch [1/5], lter [100/468], Loss: 0.4125092625617981
Train. Atk: NO_ATTACK, Epoch [1/5], lter [200/468], Loss: 0.38737356662750244
Train. Atk: NO_ATTACK, Epoch [1/5], lter [300/468], Loss: 0.4223770797252655
Train. Atk: NO_ATTACK, Epoch [1/5], lter [400/468], Loss: 0.34672805666923523
Train. Atk: NO_ATTACK, Epoch [2/5], lter [100/468], Loss: 0.29110419750213623
Train. Atk: NO_ATTACK, Epoch [2/5], lter [200/468], Loss: 0.26789724826812744
Train. Atk: NO_ATTACK, Epoch [2/5], lter [300/468], Loss: 0.4032040536403656
Train. Atk: NO_ATTACK, Epoch [2/5], lter [400/468], Loss: 0.3963630497455597
Train. Atk: NO_ATTACK, Epoch [3/5], lter [100/468], Loss: 0.3231086730957031
Train. Atk: NO_ATTACK, Epoch [3/5], lter [200/468], Loss: 0.34578952193260193
Train. Atk: NO_ATTACK, Epoch [3/5], lter [300/468], Loss: 0.1838809698820114
Train. Atk: NO_ATTACK, Epoch [3/5], lter [400/468], Loss: 0.2844764292240143
Train. Atk: NO_ATTACK, Epoch [4/5], lter [100/468], Loss: 0.21861809492

In [25]:
df = df.append(std_acc, ignore_index=True)
df

C:\Users\omen1\AppData\Local\Temp\ipykernel_6924\326607975.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(std_acc, ignore_index=True)


,NO_ATTACK,RAND_ID,FGSM_ID,PGD_ID,PGD_DCT,PGD_FFT
0,75.96+0.0,75.96+0.0,36.3+0.0,38.9+0.0,12.12+0.0,67.88+0.0


In [26]:
std_acc = get_model_avg(n_models=1, procedure='PGD_DCT')

Train. Atk: NO_ATTACK, Epoch [1/5], lter [100/468], Loss: 0.5426807403564453
Train. Atk: NO_ATTACK, Epoch [1/5], lter [200/468], Loss: 0.2903284430503845
Train. Atk: NO_ATTACK, Epoch [1/5], lter [300/468], Loss: 0.3719025254249573
Train. Atk: NO_ATTACK, Epoch [1/5], lter [400/468], Loss: 0.4022098183631897
Train. Atk: NO_ATTACK, Epoch [2/5], lter [100/468], Loss: 0.3333759903907776
Train. Atk: NO_ATTACK, Epoch [2/5], lter [200/468], Loss: 0.3562858998775482
Train. Atk: NO_ATTACK, Epoch [2/5], lter [300/468], Loss: 0.3949110209941864
Train. Atk: NO_ATTACK, Epoch [2/5], lter [400/468], Loss: 0.22702866792678833
Train. Atk: NO_ATTACK, Epoch [3/5], lter [100/468], Loss: 0.18499025702476501
Train. Atk: NO_ATTACK, Epoch [3/5], lter [200/468], Loss: 0.2900475561618805
Train. Atk: NO_ATTACK, Epoch [3/5], lter [300/468], Loss: 0.30780738592147827
Train. Atk: NO_ATTACK, Epoch [3/5], lter [400/468], Loss: 0.2132015973329544
Train. Atk: NO_ATTACK, Epoch [4/5], lter [100/468], Loss: 0.1863676458597

In [27]:
df = df.append(std_acc, ignore_index=True)
df

C:\Users\omen1\AppData\Local\Temp\ipykernel_6924\326607975.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(std_acc, ignore_index=True)


,NO_ATTACK,RAND_ID,FGSM_ID,PGD_ID,PGD_DCT,PGD_FFT
0,67.46+0.0,67.48+0.0,33.22+0.0,35.18+0.0,27.74+0.0,61.34+0.0


In [28]:
std_acc = get_model_avg(n_models=1, procedure='PGD_FFT')

Train. Atk: NO_ATTACK, Epoch [1/5], lter [100/468], Loss: 0.5452799797058105
Train. Atk: NO_ATTACK, Epoch [1/5], lter [200/468], Loss: 0.5244652032852173
Train. Atk: NO_ATTACK, Epoch [1/5], lter [300/468], Loss: 0.4158513844013214
Train. Atk: NO_ATTACK, Epoch [1/5], lter [400/468], Loss: 0.3192123472690582
Train. Atk: NO_ATTACK, Epoch [2/5], lter [100/468], Loss: 0.38677841424942017
Train. Atk: NO_ATTACK, Epoch [2/5], lter [200/468], Loss: 0.41045504808425903
Train. Atk: NO_ATTACK, Epoch [2/5], lter [300/468], Loss: 0.42866942286491394
Train. Atk: NO_ATTACK, Epoch [2/5], lter [400/468], Loss: 0.5485684275627136
Train. Atk: NO_ATTACK, Epoch [3/5], lter [100/468], Loss: 0.29808947443962097
Train. Atk: NO_ATTACK, Epoch [3/5], lter [200/468], Loss: 0.29585617780685425
Train. Atk: NO_ATTACK, Epoch [3/5], lter [300/468], Loss: 0.20634204149246216
Train. Atk: NO_ATTACK, Epoch [3/5], lter [400/468], Loss: 0.1856103241443634
Train. Atk: NO_ATTACK, Epoch [4/5], lter [100/468], Loss: 0.2325558066

In [29]:
df = df.append(std_acc, ignore_index=True)
df

C:\Users\omen1\AppData\Local\Temp\ipykernel_6924\326607975.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(std_acc, ignore_index=True)


,NO_ATTACK,RAND_ID,FGSM_ID,PGD_ID,PGD_DCT,PGD_FFT
0,87.28+0.0,84.8+0.0,16.96+0.0,3.82+0.0,0.46+0.0,69.68+0.0


In [ ]:
#Generate accuracy df/table
std_acc = get_model_avg(n_models=5, procedure="RR")

In [ ]:
df = df.append(std_acc, ignore_index=True)
df

In [ ]:
#Std accuracy
cost0=train_(model,train_loader,optimizer,loss,atks_model[0])

In [ ]:
df_acc = get_and_add_acc_model(model, test_loader, df_acc,atks_model, model_name="Std")
df_acc

In [ ]:
# Round Robin accuracy
cost_round0 = train_(model_round_rob,train_loader,optimizer_round_rob,loss,atks_model[0])
cost_round1 = round_robin(model_round_rob,train_loader,optimizer_round_rob,loss,atks_model)

In [ ]:
df_acc = get_and_add_acc_model(model_round_rob, test_loader, df_acc, atks_model, model_name="RR")
df_acc

In [ ]:
# Greedy accuracy
check_cuda_avaibilty() 
cost_greedy0 = train_(model_greedy,train_loader,optimizer_greedy,loss,atks_model[0])
cost_greedy1 = greedy(model_greedy,train_loader,valid_loader,optimizer_greedy,loss,atks_model)

In [ ]:
df_acc = get_and_add_acc_model(model_greedy, test_loader, df_acc, atks_model, model_name="Greedy")
df_acc

In [ ]:
#Multi prob. accuracy
check_cuda_avaibilty() 
cost_mult_prob0 = train_(model_mult_prob,train_loader,optimizer_mult_prob,loss,atks_model[0])
cost_mult_prob1 = multivariate_probability(model_mult_prob,train_loader,valid_loader,optimizer_mult_prob,loss,atks_model)

In [ ]:
df_acc = get_and_add_acc_model(model_mult_prob, test_loader, df_acc, atks_model, model_name="MP")
df_acc

In [ ]:
#Initial training in Pixel representation
check_cuda_avaibilty() 
cost_rand0 = train_(model_rand, train_loader, optimizer_rand, loss, atks_model[0])
cost_rand1 = train_(model_rand, train_loader, optimizer_rand, loss, atks_model[1], "RANDOM")

In [ ]:
df_acc = get_and_add_acc_model(model_rand, test_loader, df_acc, atks_model, "Rand")
df_acc

In [ ]:
#Initial training in Pixel representation
check_cuda_avaibilty() 
cost_pgd0 = train_(model_pgd, train_loader, optimizer_pgd, loss)
cost_pgd1 = train_(model_pgd, train_loader, optimizer_pgd, loss, "PGD")

In [ ]:
df_acc = get_and_add_acc_model(model_pgd, test_loader, df_acc, model_name="Pgd")
df_acc


In [ ]:
#Initial training in Pixel representation
check_cuda_avaibilty() 
cost_pgd_dct0 = train_(model_pgd_dct, train_loader, optimizer_pgd_dct, loss)
cost_pgd_dct1 = train_(model_pgd_dct, train_loader, optimizer_pgd_dct, loss, "PGD_DCT")

In [ ]:
df_acc = get_and_add_acc_model(model_pgd_dct, test_loader, df_acc, model_name="Pgd_dct")
df_acc